In [6]:
def txt_to_int(txt):
    for i in range(10):
        if txt==f'{i}':
            return i

In [8]:
from single_HMM import *
class Digt_Sequence_HMM:
    def __init__(self,sequence=None):
        self.states: List[HMMState] = []
        self.hmms=List[HMM] = []
        self.observations: List[np.ndarray] = []
        self.transitions={}
        self.state_index: Dict[HMMState, int] = {}
        self.initial_probabilities: List[float] = [] 
        self.sequence=sequence
        self.set_state_index()
    def add_digit(self,digit):
        if digit in range(0,9):
            filename='Digit {digit} HMM'
            hmm=load_hmm(filename)
            self.hmms.append(hmm)
        else:
            filename='Silence HMM'
            hmm=load_hmm(filename)
            self.hmms.append(hmm)
    def add_all_digit(self):#sequence is a 1 d list representing the recorded digit sequence e.g"187492"
        self.add_digit('Silence')
        for i in self.sequence:
            self.add_digit(i)
        self.add_digit('Silence')
    def construct_language_hmm_from_single_hmms(self):
        for hmm in self.hmms:
            for state in hmm.states:
                self.states.append(state)
        self.construct_states_topolgy_from_hmms()
        self.initialize_transitions_for_all_states()
    def recognize_the_hmm_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        state_label_integer=txt_to_int(digit_label)
        hmm=self.hmms[state_label_integer]
        return hmm
    def recognize_the_digit_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        if digit_label=="Silence":
            digit=txt_to_int(digit_label)
        else:
            digit=digit_label
        return digit
    def recognize_the_index_of_the_state_in_digit_hmm_by_label(self,state):
        full_label=state.label1
        state_index=full_label.split("-")[1]
        state_label_integer=txt_to_int(state_index)
        return state_label_integer-1
    
    def get_parent_indices(self,child_state):
        parent_states=child_state.parents
        parent_indices=[]
        for p in parent_states:
            p_hmm=self.recognize_the_hmm_the_state_belongs_to_by_label(p)
            hmm_idx=self.get_hmm_index(p_hmm)
            p_idx_in_hmm=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(p)
            p_idx=(len(p_hmm.states)-1)*hmm_idx+p_idx_in_hmm
            parent_indices.append(p_idx)
        return parent_indices
    def get_hmm_index(self,hmm):
        hmm_label=hmm.label
        for i in range(len(self.hmms)-2):
            if hmm_label==self.sequence[i]:
                return i
    def construct_states_topolgy_from_hmms(self):
        #for the first hmm
        hmm=self.hmms[0]
        #for all states except for the first state and the last state(the first state the parent is only itself and the last state is the null state)
        for j in range(1,len(hmm.states)-1):
            hmm.states[j].parents.append(hmm.states[j-1])
            hmm.states[j].parents.append(hmm.states[j])
        #for the first state, the parent is only itself
        hmm.states[0].parents.append(hmm.states[0])
        #for the rest hmms, for all hmms except for the last hmm, since in the concatenated hmm, the last state parents is only itself
        for i in range(1,len(self.hmms)-1):
            hmm=self.hmms[i]
            #for all states except for the first state and last state(the first state the parent is itself and the last state from the previous hmm, the last state is the null state)
            for j in range(1,len(hmm.states)-1):
                hmm.states[j].parents.append(hmm.states[j-1])
                hmm.states[j].parents.append(hmm.states[j])
            #for the first state
            first_state=hmm.states[0]
            #we find its previous hmm's last emission state and connect it to this state
            prev_hmm=self.hmms[i-1]
            prev_last_state=prev_hmm.states[len(prev_hmm.states)-2]
            first_state.parents.append(first_state)
            first_state.parents.append(prev_last_state)
        #for the last hmm
        hmm=self.hmms[len(self.hmms)-1]
        #for all states except for the first state and ,last emission state and last state(the first state the parent is itself and the last state from the previous hmm, the last emission state parent is only itself, the last state is the null state)
        for j in range(1,len(hmm.states)-2):
            hmm.states[j].parents.append(hmm.states[j-1])
            hmm.states[j].parents.append(hmm.states[j])
        #for the first state
        first_state=hmm.states[0]
        #we find its previous hmm's last emission state and connect it to this state
        prev_hmm=self.hmms[i-1]
        prev_last_state=prev_hmm.states[len(prev_hmm.states)-2]
        first_state.parents.append(first_state)
        first_state.parents.append(prev_last_state)
        #for the last emission state
        last_state=hmm.states[-1]
        last_state.parents.append(last_state)
    def get_first_state_idx_in_each_single_digit_hmm_train_hmm(self):
        indices=[]
        num_first_states=len(self.hmms)
        for i in range(num_first_states):
            idx=(len(self.hmms[0].states)-1)*i
            indices.append(idx)
        return indices
    def initialize_transitions_for_all_states(self):
        first_indices=self.get_first_state_idx_in_each_single_digit_hmm_train_hmm()
        for state in self.states:
            self.transitions[state]={}
        for i in range(len(self.states)):
            state=self.states[i]
            state_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(state)
            for parent in state.parents:
                p_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(parent)
                p_hmm=self.recognize_the_hmm_the_state_belongs_to_by_label(parent)
                #if current state is the first state inside the hmm
                if i==0:
                    self.transitions[parent][state]=p_hmm.transitions[p_idx][state_idx]
                #else, if current state is the first state in single digit hmms
                elif i in first_indices:
                    #if this parent is the last emission state in the previous hmm, the transition p=transition p from parent to null state in that hmm
                    if p_idx==len(self.hmms[0])-2:
                        self.transitions[parent][state]=p_hmm.transitions[p_idx][p_idx+1]
                    else:#here parent=state
                        self.transitions[parent][state]=p_hmm.transitions[p_idx][state_idx]
                else:# for the rest states
                    self.transitions[parent][state]=p_hmm.transitions[p_idx][state_idx]
    def alignment(self,seq):
        t_max=len(seq)
        num_states=len(self.states)
        Score=np.zeros((num_states,t_max))
        path = np.empty((num_states, t_max), dtype=object)
        #for the first column in DP score table score
        for i,row in enumerate(Score):
            path[i][0]=np.array([])
            if i==0:
                Score[i][0]=0
            else:
                Score[i][0]=-math.inf
        for j in range(1,t_max):
            for i,row in enumerate(Score): #i = the idx of the current node
                path[i][j]=np.array([])
                if self.states[i].isNull==False:
                    parents=self.states[i].parents
                    parent_indices=self.get_parent_indices(self.states[i])
                    max=-math.inf
                    best_parent=None
                    best_idx=None
                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        emission_score=self.states[i].log_multivariate_gaussian_pdf_diag_cov(seq[j])
                        total_score=Score[idx][j-1]+transition_score+emission_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
                else:
                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        total_score=Score[idx][j-1]+transition_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
        #for the last timestep
        max_idx=0
        max_score=-math.inf
        for i in range(num_states):
            if Score[i][t_max-1]>max_idx:
                max_idx=i
                max_score=Score[i][t_max-1]
        best_path=path[max_idx][t_max-1]
        return max_score,best_path
    def calculate_mean_and_covariance(self,vectors):
        vectors_np = np.array(vectors)
        mean_vector = np.mean(vectors_np, axis=0)
        covariance_matrix = np.cov(vectors_np.T)
        return mean_vector, covariance_matrix
    def get_clusters(self,segmented_templates):
        num_segments=len(self.states)
        a={}
        
        for i in range(len(segmented_templates)):
            for j in range(len(segmented_templates[i])):
                if i==0:
                    a[j]=np.array(segmented_templates[i][j])
                else:
                    if j>=num_segments:
                        continue
                    else:
                        if j not in a.keys():
                            a[j]=np.array(segmented_templates[i][j])
                        else:
                            a[j]=np.concatenate((a[j],np.array(segmented_templates[i][j])))
        
        return a
    def align_and_get_split(self,sample):
        max_score,best_path=self.alignment(sample)
        split_indices=self.get_split_indices(best_path)
        return split_indices
    def get_split_indices(self,hmm_states_seq):
        indices=[]
        prev_label=""
        for i,state in enumerate(hmm_states_seq):
            label=self.recognize_the_digit_the_state_belongs_to_by_label(state)
            label=str(label)
            if label!=prev_label:
                indices.append(i)
        return indices
            

    def segment_based_on_indices(self,template,indices):
        segmented_template=[]
        if len(indices)!=0:
            segmented_template.append(template[:indices[0]])
            for i in range(len(indices)-1):
                segment=template[indices[i]:indices[i+1]]
                segmented_template.append(segment)
            if indices[len(indices)-1]!=len(template):
                segmented_template.append(template[indices[len(indices)-1]:])
            else:
                segmented_template.append(template[indices[len(indices)-1]-1:])

        return segmented_template
    
def construct_digit_Sequence_HMM(seq_seq):
    HMM_sequence=[]
    for seq in seq_seq:
        HMMM=Digt_Sequence_HMM(seq)
        HMM_sequence.append(HMMM)
    return HMM_sequence
def get_sequence_name(seq_seq):
    name_sequence=[]
    for seq in seq_seq:
        name=""
        for i in seq:
            name=name+str(i)
        name=name+".npy"
        name_sequence.append(name)
    return name_sequence
def hmm_load_features(data_dir, silence_label="Silence"):
    samples = []
    for file_name in os.listdir(data_dir):
        if file_name.endswith('.npy'):
            if file_name == 'Silence.npy':
                # Load features from the 'Silence.npy' file
                features = np.load(os.path.join(data_dir, file_name))
                sample = {'label': silence_label, 'features': features}
                samples.append(sample)
            else:
                # Handle other .npy files with the expected naming convention
                parts = file_name.split('-')
                if len(parts) == 2 and parts[1].endswith('.npy'):
                    label = int(parts[0])
                    features = np.load(os.path.join(data_dir, file_name))
                    sample = {'label': label, 'features': features}
                    samples.append(sample)

    return samples

#def generate_templates(seq_name):


In [10]:
seq_seq=[[0,1,2,3,4,5,6,7,8,9],[9,8,7,6,5,4,3,2,1,0],[1,2,3,4,5,6,7,8,9,0],[0,9,8,7,6,5,4,3,2,1],[1,3,5,7,9,0,2,4,6,8],[8,6,4,2,0,9,7,5,3,1]]
seq_name=get_sequence_name(seq_seq)
#load all templates
HMM_seq=construct_digit_Sequence_HMM(seq_seq)
def train(HMM_seq,templates):
    for HMM in HMM_seq:
        

    

TypeError: '_SpecialGenericAlias' object does not support item assignment

In [ ]:
#This is the class we built for Proj5's second problem, we only have one null state at the beginning in the recognizer
class unrestricted_recognizer:
    def __init__(self,hmms):
        self.hmms=hmms
        self.states=[]
        self.construct_states_from_hmms()
        self.initial_probability=1/len(self.hmms)
        self.transitions={}
        self.initialize_transitions()
    #suppose the first state in list self.hmms is a null state
    def construct_states_from_hmms(self):
        #add a null state in the beginning
        ns=HMMState(label1="Null State",isNull=True)
        self.states.append(ns)
        for hmm in self.hmms:
            #-1 because when constructing the hmm for the recognizer we don't want the null state in each digit hmm to come in
            for i in range(len(hmm.states)-1):
                state=hmm.states[i]
                full_label=state.label1
                
                state_label=full_label.split("-")[1]
                state_label_integer=txt_to_int(state_label)
                #if the state is the first state in the digit hmm, the parent is the null state and itself
                if state_label_integer==1:
                    state.parents.append(state)
                    state.parents.append(self.states[0])#null state
                else:#else, the parent is its previous state in the digit hmm and also itself
                    state.parents.append(hmm.states[state_label_integer-2])
                    state.parents.append(state)
                self.states.append(state)
        for hmm in self.hmms:
            #for the null state in the recognizer, its parents are the last state in digit hmms
            self.states[0].parents.append(hmm.states[-1])
    def initialize_transitions(self):
        #Transition is represented in dictionary, the key is HMMState, value is also dictionary, so self.transitions[parent][state] is the transition probability from parent to current state
        for state in self.states:
            self.transitions[state]={}
        for state in self.states:
            #if the state is null state,#his set the transition p from  emitting states to the null state
            if state.isNull==True:
                for parent in state.parents:
                    #find the hmm the parent belong to
                    hmm=self.recognize_the_hmm_the_state_belongs_to_by_label(parent)
                    #find the index of the parent in the hmm
                    parent_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(parent)
                    self.transitions[parent][state]=hmm.transitions[parent_idx][parent_idx+1]#This is because in a digit hmm, we have 6 states, and the last state is null state
            #else
            else:
                for parent in state.parents:
                    #if the state's parent is null state, the the transitioin p from the parent to the state is 1/len(self.hmms)
                    if parent.isNull==True:#This set the transition p from  null state to the emitting state
                        self.transitions[parent][state]=1/len(self.hmms)
                    else:
                        #if the state and its parent are all emitting states, find the hmm model they belongs to
                        hmm=self.recognize_the_hmm_the_state_belongs_to_by_label(parent)
                        state_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(state)
                        parent_idx=self.recognize_the_index_of_the_state_in_digit_hmm_by_label(parent)
                        self.transitions[parent][state]=hmm.transitions[parent_idx][state_idx]
    def recognize_the_hmm_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        state_label_integer=txt_to_int(digit_label)
        hmm=self.hmms[state_label_integer]
        return hmm
    def recognize_the_digit_the_state_belongs_to_by_label(self,state):
        full_label=state.label1
        digit_label=full_label.split("-")[0]
        digit=txt_to_int(digit_label)
        return digit
    def recognize_the_index_of_the_state_in_digit_hmm_by_label(self,state):
        full_label=state.label1
        state_index=full_label.split("-")[1]
        state_label_integer=txt_to_int(state_index)
        return state_label_integer-1
    def get_parent_indices(self,child_state):
        
        parent_states=child_state.parents
        parent_indices=[]
        for p in parent_states:
            label=p.label1
            digit=txt_to_int(label.split("-")[0])
            rank=txt_to_int(label.split("-")[1])
            idx=len(self.hmms[0].states)*digit+rank
            parent_indices.append(idx)
        return parent_indices
    def print_best_path(best_path):
        str=""
        for state in best_path:
            label=state.label1
            digit=label.split("-")[0]
            str.append(digit)
        print(f'The best path is: {str}')
    def get_first_state_idx_in_each_single_digit_hmm_in_reconizer_hmm(self):
        indices=[]
        num_first_states=len(self.hmms)
        for i in range(num_first_states):
            idx=(len(self.hmms[0].states)-1)*i+1
            indices.append(idx)
        return indices
    def alignment(self,seq):
        t_max=len(seq)
        num_states=len(self.states)
        Score=np.zeros((num_states,t_max))
        path = np.empty((num_states, t_max), dtype=object)
        first_state_indices=self.get_first_state_idx_in_each_single_digit_hmm_in_reconizer_hmm()
        #for the first column in DP score table score
        for i,row in enumerate(Score):
            #can only start at the null state
            if self.states[i].isNull==False:
                if i in first_state_indices:
                    Score[i][0]=math.log(self.initial_probability)
                else:
                    Score[i][0]=-math.inf
            else:
                Score[i][0]=0
         #for the first column in DP path table score
        for i,row in enumerate(path):
            #can only start at the null state
            path[i][0]=np.array([])
        for j in range(1,t_max):
            for i,row in enumerate(Score): #i = the idx of the current node
                path[i][j]=[]
                if self.states[i].isNull==False:
                    parents=self.states[i].parents
                    parent_indices=self.get_parent_indices(self.states[i])
                    max=-math.inf
                    best_parent=None
                    best_idx=None
                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        emission_score=self.states[i].log_multivariate_gaussian_pdf_diag_cov(seq[j])
                        total_score=Score[idx][j-1]+transition_score+emission_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
                else:
                    for idx in parent_indices:
                        transition_p=self.transitions[idx][i]
                        transition_score=math.log(transition_p)
                        total_score=Score[idx][j-1]+transition_score
                        if total_score>max:
                            max=total_score
                            best_parent=parents[idx]
                            best_idx=idx
                    Score[i][j]=max
                    path[i][j]=np.concatenate((path[best_idx][j-1],np.array([best_parent])))
        #for the last timestep
        max_idx=0
        max_score=-math.inf
        for i in range(num_states):
            if Score[i][t_max-1]>max_idx:
                max_idx=i
                max_score=Score[i][t_max-1]
        best_path=path[max_idx][t_max-1]
        self.print_best_path(best_path)
        return max_score,best_path
    def recognize(self,seq):
        max_score,best_path=self.alignment(seq)
        digit_seq=''
        prev_digit=-1
        for state in best_path:
            digit=self.recognize_the_digit_the_state_belongs_to_by_label(state)
            if digit!=prev_digit:
                digit_seq=digit_seq+str(digit)
                prev_digit=digit
        print(f'The number is recognized as {digit_seq}')
def print_transition_information(recognizer,i):
    key, value = list(recognizer.transitions.items())[i]
    f=key.label1
    print(f'Transit From {key}')
    print(f'Transit to {value}')
    print(f'The State Label is:{f}')

In [ ]:
hmmss=load_all_hmm()

HMM model has been loaded from 'Digit 0 HMM'
HMM model has been loaded from 'Digit 1 HMM'
HMM model has been loaded from 'Digit 2 HMM'
HMM model has been loaded from 'Digit 3 HMM'
HMM model has been loaded from 'Digit 4 HMM'
HMM model has been loaded from 'Digit 5 HMM'
HMM model has been loaded from 'Digit 6 HMM'
HMM model has been loaded from 'Digit 7 HMM'
HMM model has been loaded from 'Digit 8 HMM'
HMM model has been loaded from 'Digit 9 HMM'


In [ ]:
recognizer=unrestricted_recognizer(hmms=hmmss)


AttributeError: 'HMMState' object has no attribute 'parents'

In [ ]:
print_transition_information(recognizer,1)

Transit From <single_HMM.HMMState object at 0x00000293753E8650>
Transit to {<single_HMM.HMMState object at 0x00000293753E8650>: 0.9393939393939394, <single_HMM.HMMState object at 0x00000293753E8E90>: 0.06060606060606061}
The State Label is:0-1


In [ ]:
import numpy as np

# Define the dimensions of the array
n = 3  # Number of rows
m = 4  # Number of columns

# Initialize the 2D array with empty lists
empty_array = np.empty((n, m), dtype=object)

# Print the initialized array

empty_array


array([[None, None, None, None],
       [None, None, None, None],
       [None, None, None, None]], dtype=object)